<a href="https://colab.research.google.com/github/Anwesha-code/PBL_Project/blob/main/RandomForestCEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_center_carbon_emission_dataset_COPY.xlsx to data_center_carbon_emission_dataset_COPY.xlsx


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_excel('data_center_carbon_emission_dataset_COPY.xlsx')
print(df.shape)
print(df.count())

(75000, 18)
MAC                  75000
weekday              75000
timestamp            75000
voltage              75000
current              75000
power                75000
frequency            75000
energy               75000
power_factor         75000
esp32_temperature    75000
cpu_temperature      75000
gpu_temperature      75000
cpu_usage_percent    75000
cpu_power_watts      75000
gpu_usage_percent    75000
gpu_power_watts      75000
ram_usage_percent    75000
ram_power_watts      75000
dtype: int64


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np

# Load data
df = pd.read_excel('data_center_carbon_emission_dataset_COPY.xlsx')

# Convert timestamp to datetime and chronological sort
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Handle missing values
df.ffill(inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Feature engineering: hour and day of week
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek

# Outlier removal (z-score, 3-sigma rule)
z_scores = np.abs(stats.zscore(df.select_dtypes(include=[float, int])))
df = df[(z_scores < 3).all(axis=1)]

# Scaling numerical features
num_cols = [
    'voltage', 'current', 'power', 'frequency', 'energy', 'power_factor',
    'esp32_temperature', 'cpu_temperature', 'gpu_temperature',
    'cpu_usage_percent', 'cpu_power_watts', 'gpu_usage_percent',
    'gpu_power_watts', 'ram_usage_percent', 'ram_power_watts'
]
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Categorical encoding with one-hot (MAC, weekday)
df = pd.get_dummies(df, columns=['MAC', 'weekday'], drop_first=True)

# Check for missing values and duplicates (again, after all steps)
print("Missing values per column:\n", df.isnull().sum())
df = df.drop_duplicates()

# Verify sum of one-hot columns for weekdays is 0 or 1 per row (because of drop_first=True)
weekday_cols = [col for col in df.columns if col.startswith('weekday_')]
assert (((df[weekday_cols].sum(axis=1) == 1) | (df[weekday_cols].sum(axis=1) == 0)).all()), "Error: Weekday encoding not correct"

# Temporal lag features (e.g., for voltage)
df = df.sort_values('timestamp').reset_index(drop=True)
df['voltage_lag1'] = df['voltage'].shift(1)

# Drop NA values from lag creation (first row)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Chronological split for train, validation, test (70/15/15 split)
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)
train = df[:train_size]
val = df[train_size:train_size + val_size]
test = df[train_size + val_size:]

print("Train shape:", train.shape)
print("Validation shape:", val.shape)
print("Test shape:", test.shape)
print("First 5 rows:\n", train.head())


Missing values per column:
 timestamp                0
voltage                  0
current                  0
power                    0
frequency                0
energy                   0
power_factor             0
esp32_temperature        0
cpu_temperature          0
gpu_temperature          0
cpu_usage_percent        0
cpu_power_watts          0
gpu_usage_percent        0
gpu_power_watts          0
ram_usage_percent        0
ram_power_watts          0
hour                     0
dayofweek                0
MAC_4D:62:06:13:97:31    0
MAC_5E:63:07:14:98:32    0
MAC_6F:64:08:15:99:33    0
weekday_1                0
weekday_2                0
weekday_3                0
weekday_4                0
weekday_5                0
weekday_6                0
dtype: int64
Train shape: (51961, 28)
Validation shape: (11134, 28)
Test shape: (11135, 28)
First 5 rows:
             timestamp   voltage   current     power  frequency    energy  \
0 2025-08-23 00:00:30 -0.143929 -1.067405 -0.920441  -0.0075

Timestamp parsing & sorting, which is vital for time series

Handling missing values and duplicates

Feature engineering: cyclical splits, lag features, rolling means

Outlier removal (z-score based)

Scaling all numeric features (including engineered ones)

One-hot encoding for categorical data

Checks for cleanliness after all preprocessing

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Assuming df is preprocessed and split chronologically into train, val, test as before:
# Features and target selection (for example, predict 'energy')
target_col = 'energy'
feature_cols = df.columns.drop(['timestamp', 'energy'])  # drop timestamp & target from features

# Prepare train, val, test
X_train = train[feature_cols]
y_train = train[target_col]

X_val = val[feature_cols]
y_val = val[target_col]

X_test = test[feature_cols]
y_test = test[target_col]

# Initialize and train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict on validation and test sets
y_val_pred = rf.predict(X_val)
y_test_pred = rf.predict(X_test)

# Evaluate
val_mae = mean_absolute_error(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Validation MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}')
print(f'Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}')


Validation MAE: 1.5321, RMSE: 1.5495
Test MAE: 1.9634, RMSE: 1.9753
